In [1]:
import pandas as pd

In [7]:
preference_text= (
        "quick, savoury week-night dinner that is high protein, low carb, "
        "dairy-free and ready in under 30 minutes")


pref = preference_text.lower()

# --- Step 1: Parse preferences from text ---
preferences = {
    'low_carb': any(kw in pref for kw in ['low carb', 'low carb', 'keto']),
    'high_protein': any(kw in pref for kw in ['high protein', 'protein-rich']),
    'low_fat': any(kw in pref for kw in ['low fat', 'low-fat']),
    'dairy_free': any(kw in pref for kw in ['dairy-free', 'no dairy']),
    'vegetarian': 'vegetarian' in pref,
    'vegan': 'vegan' in pref,
    'gluten_free': 'gluten-free' in pref or 'no gluten' in pref,
}

# Thresholds
if preferences['low_carb']:
    preferences['max_energy'] = 500
if preferences['high_protein']:
    preferences['min_protein'] = 20
if preferences['low_fat']:
    preferences['max_fat'] = 10

preferences

{'low_carb': True,
 'high_protein': True,
 'low_fat': False,
 'dairy_free': True,
 'vegetarian': False,
 'vegan': False,
 'gluten_free': False,
 'max_energy': 500,
 'min_protein': 20}

In [8]:
recipe_data_path = "full_recipe_embeddings.pkl"
filtered = pd.read_pickle(recipe_data_path)[['id',
                           'title',
                           'clean_instructions',
                           'instructions_full',
                           'ingredients_clean',
                           'ingredients_embedding',
                           'full_embedding',
                           'nutrition_per_100g']]

filtered.head()

,id,title,clean_instructions,instructions_full,ingredients_clean,ingredients_embedding,full_embedding,nutrition_per_100g
0,000095fc1d,Yogurt Parfaits,layer all ingredients in a serving dish.,Layer all ingredients in a serving dish.,"[non - fat vanilla yogurt, strawberries, low -...","[-0.023122193291783333, -0.04638631269335747, ...","[-0.01155478972941637, -0.05617685988545418, 0...","{'energy': 81.12946131894766, 'fat': 2.1401392..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",cream sugar and butter together till smooth. a...,Cream sugar and butter together till smooth. A...,"[granulated sugar, corn oil, Egg Beaters egg s...","[-0.04340571165084839, -0.02153887413442135, 0...","[-0.03859461471438408, 0.0001314176624873653, ...","{'energy': 477.09640393594606, 'fat': 23.41248..."
2,00059b093b,Honey Sriracha Chicken Wings,preheat oven to 400 degrees f. in a large bowl...,Preheat oven to 400 degrees F. In a large bowl...,"[chicken wings, salt, pepper, oil, butter, hot...","[-0.04409683123230934, -0.0222036000341177, 0....","[-0.011450769379734993, -0.03441989794373512, ...","{'energy': 208.05898280960727, 'fat': 14.29704..."
3,0005fc89f7,Shrimp and Caper Salad,"in a large bowl, toss the shrimp, green onions...","In a large bowl, toss the shrimp, green onions...","[shrimp, green onions, celery, capers, black o...","[-0.04525844380259514, -0.034785542637109756, ...","[-0.09913121163845062, 0.001413632300682366, 0...","{'energy': 194.7525956849092, 'fat': 15.980767..."
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,measure out the cocoa powder into a mixing bow...,Measure out the cocoa powder into a mixing bow...,"[cocoa powder, honey, natural - style peanut b...","[-0.05007326230406761, -0.023665567860007286, ...","[-0.02296869456768036, 0.007631730753928423, 0...","{'energy': 303.43539955190437, 'fat': 5.094846..."


In [9]:
filtered['nutrition_per_100g'][0]

{'energy': 81.12946131894766,
 'fat': 2.140139263515891,
 'protein': 6.914436593565536,
 'salt': 0.05597816738985967,
 'saturates': 0.36534716195613937,
 'sugars': 5.08634103436144}

In [11]:
new_filtered = filtered[
                filtered['nutrition_per_100g'].apply(
                    lambda x: x.get('energy') < preferences['max_energy']
                )
            ]

new_filtered = filtered[
                filtered['nutrition_per_100g'].apply(
                    lambda x: x.get('protein', 0) > preferences['min_protein']
                )
            ]

new_filtered

,id,title,clean_instructions,instructions_full,ingredients_clean,ingredients_embedding,full_embedding,nutrition_per_100g
74,0057bc5426,Baked Cheesecake,cream the cream cheese in mixer. add sugar and...,Cream the cream cheese in mixer. Add sugar and...,"[cream cheese, fine sugar, eggs, fresh lemon j...","[-0.05195024609565735, -0.03514965996146202, 0...","[0.0010216707596555352, -0.04644177481532097, ...","{'energy': 368.60145781969254, 'fat': 17.22226..."
131,009365841b,Ethiopian Spicy Cheese Dip,"preparation:. in a bowl, flake the cheese with...","Preparation:. In a bowl, flake the cheese with...","[farmer cheese, clarified butter, cayenne pepp...","[-0.06259503215551376, -0.039961013942956924, ...","[-0.05907848849892616, 0.033154964447021484, -...","{'energy': 391.27578482216836, 'fat': 25.03544..."
183,00cd7e9969,Basil Cheese Torte,line a round bowl with waxed paper. using pest...,Line a round bowl with waxed paper. Using pest...,"[fresh basil, fresh parsley, parmesan cheese, ...","[-0.04874924197793007, -0.042481645941734314, ...","[-0.10263366252183914, -0.01745292916893959, 0...","{'energy': 494.79554337436076, 'fat': 43.82521..."
194,00dacdee49,Shrimp Cocktail,bring a large pot of water to the boil and boi...,Bring a large pot of water to the boil and boi...,[shrimp],"[-0.0262391846626997, -0.0005726515082642436, ...","[-0.03532426059246063, 0.07825817167758942, -0...","{'energy': 85.00000000000001, 'fat': 0.51, 'pr..."
215,00ff556727,How-to Roast Almonds Recipe elanaspantry,spread the almonds out in two 9x13 inch baking...,Spread the almonds out in two 9x13 inch baking...,"[almonds, sea salt]","[-0.018622688949108124, -0.0059594763442873955...","[0.051782988011837006, -0.007643986027687788, ...","{'energy': 571.441121690543, 'fat': 49.2781609..."
...,...,...,...,...,...,...,...,...
51061,ff1e8f8beb,Tasso Cajun Style,trim the pork of all excess fat and cut it int...,Trim the pork of all excess fat and cut it int...,"[pork butt, salt, cayenne pepper, black pepper...","[-0.0761210173368454, -0.03206101432442665, -0...","[-0.011315898038446903, 0.05440032482147217, 0...","{'energy': 111.2146122572918, 'fat': 2.2444597..."
51072,ff2f1bcaf4,Dried Beef Cheese Ball Recipe,"blend with hands and form into a ball, chill.","Blend with hands and form into a ball, chill.","[beef, cream cheese, parmesan cheese, pimiento...","[-0.02284778282046318, -0.0440092459321022, 0....","[-0.05641312524676323, -0.05719146877527237, 0...","{'energy': 317.3062928278164, 'fat': 21.055675..."
51076,ff3268b961,Hot Artichoke and Spinach Dip,boil the spinach and artichoke hearts in a cup...,Boil the spinach and artichoke hearts in a cup...,"[artichoke hearts, frozen chopped spinach, lig...","[-0.05171375721693039, -0.058431897312402725, ...","[0.018341360613703728, -0.03733081743121147, -...","{'energy': 306.61263201640924, 'fat': 19.57466..."
51103,ff560c15b9,Almond Poppy Noodles!,cook noodles according to package directions. ...,Cook noodles according to package directions. ...,"[egg noodles, butter, slivered almonds, poppy ...","[-0.04661644995212555, 0.011552143841981888, 0...","[-0.085317462682724, -0.011890155263245106, 0....","{'energy': 515.5419151792591, 'fat': 31.566641..."


In [2]:
import json
import pandas as pd
# -- df_layer1: from layer1.json --
with open("recipe1m_downloads/extracted/layer1.json", "r", encoding="utf-8") as f:
    layer1_data = json.load(f)

df_layer1 = pd.DataFrame([
    {
        "id": r["id"],
        "title": r["title"],
        "partition": r["partition"],
        "url": r.get("url"),
        "ingredients_raw": [i["text"] for i in r.get("ingredients", [])],
        "instructions_full": " ".join([s["text"] for s in r.get("instructions", [])])
    }
    for r in layer1_data
])
df_layer1.head(1)

,id,title,partition,url,ingredients_raw,instructions_full
0,000018c8a5,Worlds Best Mac and Cheese,train,http://www.epicurious.com/recipes/food/views/-...,"[6 ounces penne, 2 cups Beechers Flagship Chee...",Preheat the oven to 350 F. Butter or oil an 8-...


In [3]:
with open("recipe1m_downloads/det_ingrs.json", "r", encoding="utf-8") as f:
    det_ingrs_data = json.load(f)

df_det_ingrs = pd.DataFrame([
    {
        "id": r["id"],
        "ingredients_clean": [ing["text"] for ing, valid in zip(r["ingredients"], r["valid"]) if valid]
    }
    for r in det_ingrs_data
])

def parse_full_nutrition_entry(entry):
    n = len(entry["ingredients"])

    ingredients_structured = []
    for i in range(n):
        ingredients_structured.append({
            "ingredient": entry["ingredients"][i]["text"] if i < len(entry["ingredients"]) else None,
            "quantity": entry["quantity"][i]["text"] if i < len(entry["quantity"]) else None,
            "unit": entry["unit"][i]["text"] if i < len(entry["unit"]) else None,
            "weight_g": entry["weight_per_ingr"][i] if i < len(entry["weight_per_ingr"]) else None,
            "nutrition": entry["nutr_per_ingredient"][i] if i < len(entry["nutr_per_ingredient"]) else None
        })

    return {
        "id": entry.get("id"),
        "title": entry.get("title"),
        "partition": entry.get("partition"),
        "url": entry.get("url"),
        "instructions": " ".join([step["text"] for step in entry.get("instructions", [])]),
        "ingredients_structured": ingredients_structured,
        "nutrition_per_100g": entry.get("nutr_values_per100g", {}),
        "fsa_lights_per_100g": entry.get("fsa_lights_per100g", {})
    }

df_det_ingrs

,id,ingredients_clean
0,000018c8a5,"[penne, cheese sauce, cheddar cheese, gruyere ..."
1,000033e39b,"[elbow macaroni, American cheese, celery, gree..."
2,000035f7ed,"[tomatoes, kosher salt, red onion, green bell ..."
3,00003a70b1,"[milk, water, butter, mashed potatoes, whole k..."
4,00004320bb,"[watermelon gelatin, boiling water, Cool Whip,..."
...,...,...
1029715,ffffbb45d2,"[butter, sugar, eggs, dark rum, lemon zest, sa..."
1029716,ffffcd4444,"[daikon radishes, sesame oil, white sesame see..."
1029717,ffffd33513,"[apple cider, sugar, kosher salt, bay leaves, ..."
1029718,ffffd533d7,"[ground veal, sweet Italian sausage, dry bread..."


In [4]:
# Load your JSON file or individual object here
with open("recipe1m_downloads/recipes_with_nutritional_info.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Example: parse just the first entry (you can loop later)
parsed = [parse_full_nutrition_entry(recipe) for recipe in data]

# Convert to DataFrame
df_nutrition_full = pd.DataFrame(parsed)

df_nutrition_full.head()

,id,title,partition,url,instructions,ingredients_structured,nutrition_per_100g,fsa_lights_per_100g
0,000095fc1d,Yogurt Parfaits,train,http://tastykitchen.com/recipes/breakfastbrunc...,Layer all ingredients in a serving dish.,"[{'ingredient': 'yogurt, greek, plain, nonfat'...","{'energy': 81.12946131894766, 'fat': 2.1401392...","{'fat': 'green', 'salt': 'green', 'saturates':..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",val,http://cookeatshare.com/recipes/salt-free-low-...,Cream sugar and butter together till smooth. A...,"[{'ingredient': 'sugars, granulated', 'quantit...","{'energy': 477.09640393594606, 'fat': 23.41248...","{'fat': 'red', 'salt': 'orange', 'saturates': ..."
2,00059b093b,Honey Sriracha Chicken Wings,train,http://tastykitchen.com/recipes/main-courses/h...,Preheat oven to 400 degrees F. In a large bowl...,"[{'ingredient': 'chicken, broilers or fryers, ...","{'energy': 208.05898280960727, 'fat': 14.29704...","{'fat': 'orange', 'salt': 'orange', 'saturates..."
3,0005fc89f7,Shrimp and Caper Salad,train,http://allrecipes.com/recipe/shrimp-and-caper-...,"In a large bowl, toss the shrimp, green onions...","[{'ingredient': 'crustaceans, shrimp, raw (not...","{'energy': 194.7525956849092, 'fat': 15.980767...","{'fat': 'orange', 'salt': 'orange', 'saturates..."
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,train,http://www.food.com/recipe/natural-peanut-butt...,Measure out the cocoa powder into a mixing bow...,"[{'ingredient': 'cocoa, dry powder, unsweetene...","{'energy': 303.43539955190437, 'fat': 5.094846...","{'fat': 'orange', 'salt': 'green', 'saturates'..."


In [6]:
print(len(df_layer1))

1029720


In [ ]:
# Step 1: merge layer1 and det_ingrs
df_combined = pd.merge(df_layer1, df_det_ingrs, on="id", how="inner")

# Step 2: merge with nutrition
df_final = pd.merge(df_combined, df_nutrition_full, on="id", how="inner", suffixes=("", "_nutrition"))
drop_columns = ['partition','url', 'partition_nutrition']
df_final_filtered = df_final.drop(drop_columns, axis = 1)

df_final_filtered